In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Crew, Task

In [3]:
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [6]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [7]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [8]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [9]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [10]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [11]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [12]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [13]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [14]:
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

In [15]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [16]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I must consider the theme, size, and budget constraints of the event.

Action: Search the internet
Action Input: {"search_query": "venues for events in San Francisco"} 


Search results: Title: Top 20 Venues in San Francisco, CA - PartySlate
Link: https://www.partyslate.com/find-venues/near/san-francisco-ca-usa
Snippet: The San Francisco Ferry Building, located right on the waterfront, offers a one-of-a-kind venue for special events—from weddings to corporate receptions.
---
Title: Small Party Venues for Rent in San Francisco, CA - Tagvenue
Link: https://www.tagvenue.com/us/hire/small-party-venues/san-francisco
Snippet: San Francisco districts that offer attractive event venues for small part

This is the agent final answer: Venue: City View at METREON
Location: SoMa
Capacity: 2,000
Details: 
City View at Metreon in San Francisco is a versatile event venue. Newly renovated to offer both flexibility and upscale ambiance, this San Francisco event space features more than 31,000 square feet of space that can be adjusted to accommodate various events, ranging from intimate gatherings of 200 to large-scale events for up to 2,000 people. 
Its key attractions include floor-to-ceiling windows providing an unobstructed view of San Francisco’s skyline and an expansive outdoor terrace, offering a unique open-air experience. This makes it an ideal venue for receptions, conferences, and other special occasions.
Please provide a feedback:  Yes


Final Answer: 
Venue: City View at METREON
Location: SoMa
Capacity: 2,000
Details: 
City View at Metreon in San Francisco is a versatile event venue. Newly renovated to offer both flexibility and upscale ambiance, this San Francisco event space features more than 31,000 square feet of space that can be adjusted to accommodate various events, ranging from intimate gatherings of 200 to large-scale events for up to 2,000 people. 
Its key attractions include floor-to-ceiling windows providing an unobstructed view of San Francisco’s skyline and an expansive outdoor terrace, offering a unique open-air experience. This makes it an ideal venue for receptions, conferences, and other special occasions.

> Finished chain.
 [DEBUG]: == [Venue Coordinator] Task output: {
  "name": "City View at METREON",
  "address": "SoMa",
  "capacity": 2000,
  "booking_status": "Available"
}


 [DEBUG]: == Working Agent: Logistics Manager
 [INFO]: == Starting Task: Coordinate catering and equipment for an event 

Exception in thread Thread-6 (_execute):
Traceback (most recent call last):
  File "/Users/ericmelz/.pyenv/versions/3.12.0/lib/python3.12/threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "/Users/ericmelz/Data/code/crewai-course/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/ericmelz/.pyenv/versions/3.12.0/lib/python3.12/threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ericmelz/Data/code/crewai-course/venv/lib/python3.12/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/Users/ericmelz/Data/code/crewai-course/venv/lib/python3.12/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ericmelz/Data/code/crewai-course/venv/lib/python3.12/site-packages/langchain/chains/base

In [17]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'SoMa',
 'booking_status': 'Available',
 'capacity': 2000,
 'name': 'City View at METREON'}
